## Lending club default rate prediction using R
----
1. The dataset is from lending club official site. (https://www.lendingclub.com/info/download-data.action)  
2. These files contain complete loan data for all loans issued through the time period stated, including the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. The file containing loan data through the "present" contains complete loan data for all loans issued through the previous completed calendar quarter.  
3. I choose 2016Q1 since this will let the loan more time to develop. If you compare with 2018Q1 data, you will find that most of the 2018Q1 data is under 'current' category because these loans are just issued. 
4. There is another LCDataDictionary.xlsx file on the website to explain each feature in the data set. 

In [31]:
loan <- read.csv('./LoanStats_securev1_2016Q1.csv', header = TRUE, stringsAsFactors = FALSE, skip = 1)

In [ ]:
# loanBackUp <- loan

In [32]:
library(zoo)


Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



In [33]:
# take a look at the loan data, there are many colunms not shown in here.
head(loan)

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,⋯,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
76003542,NA,16000,16000,16000,36 months,5.32%,481.84,A,A1,⋯,NA,NA,Cash,N,,,,NA,NA,NA
76023477,NA,17000,17000,17000,36 months,9.75%,546.55,B,B3,⋯,NA,NA,Cash,N,,,,NA,NA,NA
75993535,NA,15000,15000,15000,60 months,15.31%,359.30,C,C5,⋯,NA,NA,Cash,N,,,,NA,NA,NA
73458582,NA,2425,2425,2425,36 months,15.31%,84.44,C,C5,⋯,NA,NA,Cash,N,,,,NA,NA,NA
73511090,NA,29900,29900,29900,36 months,12.99%,1007.31,C,C2,⋯,NA,NA,Cash,N,,,,NA,NA,NA
75304067,NA,12000,12000,12000,36 months,11.99%,398.52,C,C1,⋯,NA,NA,Cash,N,,,,NA,NA,NA


In [34]:
# Issue date is all in 2016Q1
# Notice that there are some missing values in next_pymnt_d column. 
# This is because these ones are already paid off or charged off.
# The date format is not supported by R. We can use zoo lib to transfer them.
head(loan[, c('issue_d', 'last_pymnt_d', 'next_pymnt_d')])

issue_d,last_pymnt_d,next_pymnt_d
Mar-2016,May-2016,
Mar-2016,Nov-2017,Dec-2017
Mar-2016,Apr-2017,
Mar-2016,Dec-2016,
Mar-2016,Sep-2017,Dec-2017
Mar-2016,Jul-2016,


In [35]:
# take a closer look at the data where next_pymnt_d is empty 
dim(subset(loan, next_pymnt_d == ""))
with(subset(loan, next_pymnt_d == ""), table(loan_status)) # either charged off or fully paid
with(subset(loan, next_pymnt_d == "" & last_pymnt_d == ""), table(loan_status)) # all charged off

[1] 48695   151

loan_status
            Charged Off  Fully Paid 
          2       13043       35650 

loan_status
            Charged Off 
          2         148 

In [36]:
# Noticed that there are 2 N.A. in the dataset. Let's take a look at them
subset(loan, loan_status == "")

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,⋯,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
133888,Total amount funded in policy code 1: 2087217200,NA,NA,NA,NA,,,NA,,,⋯,NA,NA,,,,,,NA,NA,NA
133889,Total amount funded in policy code 2: 662815446,NA,NA,NA,NA,,,NA,,,⋯,NA,NA,,,,,,NA,NA,NA


In [37]:
# As is seen, these two data points are NULL. We can delte them from data sets.
loan <- subset(loan, loan_status != "")

In [38]:
# where last_pymnt_d is empty
with(subset(loan, last_pymnt_d == ""), table(loan_status))

loan_status
Charged Off 
        148 

In [39]:
# In summary, the next_pymnt_d and last_pymnt_d can give us four combinations depending on their states.

# all charged off
table(subset(loan, next_pymnt_d == "" & last_pymnt_d == "")$loan_status)
# charged off or fully paid
table(subset(loan, next_pymnt_d == "" & last_pymnt_d != "")$loan_status)
# doesn't exist this situation where we don't have last_pymnt_d but have next_pymnt_d
table(subset(loan, next_pymnt_d != "" & last_pymnt_d == "")$loan_status)
# normal situation, could be late payment but is still paying
table(subset(loan, next_pymnt_d != "" & last_pymnt_d != "")$loan_status)


Charged Off 
        148 


Charged Off  Fully Paid 
      12895       35650 

< table of extent 0 >


           Current            Default    In Grace Period  Late (16-30 days) 
             79931                 11               1637                458 
Late (31-120 days) 
              3157 

In [40]:
# change the date format to R supported format.
loan$issue_d_1 <- as.Date(as.yearmon(loan$issue_d, "%b-%Y"))
loan$issue_year <- as.character(format(loan$issue_d_1, "%Y"))
loan$issue_mon <- as.character(format(loan$issue_d_1, "%m"))

loan$last_pymnt_d_1 <- as.Date(as.yearmon(loan$last_pymnt_d, "%b-%Y"))
loan$last_pymnt_year <- as.character(format(loan$last_pymnt_d_1, "%Y"))
loan$last_pymnt_mon <- as.character(format(loan$last_pymnt_d_1, "%m"))

In [41]:
# Based on the observations above, 
# one question I want to explore is that at which phase does the loan more likely to be missed. 
# I can create a new feature that shows how long the pymnt has been made. 

loan$last_pymnt_from_issue <- with(loan, last_pymnt_d_1 - issue_d_1)
table(loan$last_pymnt_from_issue)


    0    29    31    60    61    90    91    92   121   122   151   152   153 
  492   455  1071   786   636   430   339   698  1028   915   659   509   979 
  182   184   213   214   243   244   245   274   275   304   305   306   335 
 1263  1035  1337  1135   728   661  1149  1288  1144   730   614  1257  1553 
  337   365   366   394   396   397   425   426   455   456   457   486   487 
 1158  1665  1411  1007  1336   615  1709  1353   863   581  1372  1595  1399 
  516   517   518   547   549   578   579   608   609   610   639   670 
  905   753  1499  1616  1306  1627  1947  1280   693 39013 25046 19099 

In [42]:
# As we can see above, there are different days meaning the loan has been paid for 1month, 2 months, ... 1year ... 2years, etc.
# we can categorize this into below 90 days, below half year, below 270 days, etc.

loan$last_pymnt_from_issue_cat <- with(loan, as.character(cut(as.numeric(last_pymnt_from_issue), 
                                                              c(-1, 0, 92, 184, 275, 366, 457, 549, 639, 730))))

In [43]:
# take a look at the distribution of it
table(loan$last_pymnt_from_issue_cat)


   (-1,0]    (0,92] (184,275] (275,366] (366,457] (457,549] (549,639] (639,730] 
      492      4415      7442      8388      8836      9073     69606     19099 
 (92,184] 
     6388 

In [44]:
# Remember the 148 NA from last_pymnt_d ? table doesn't take the NA into account.
# notice that the latter two added up to the first one.
dim(loan)
sum(table(loan$last_pymnt_from_issue_cat))
table(subset(loan, is.na(last_pymnt_from_issue_cat))$last_pymnt_d)

[1] 133887    159

[1] 133739


    
148 

In [45]:
# create a category for these NA values as no pymnt
loan$last_pymnt_from_issue[which(is.na(loan$last_pymnt_from_issue))] <- 2000
loan$last_pymnt_from_issue_cat[which(is.na(loan$last_pymnt_from_issue_cat))] <- 'no pymnt'

In [46]:
# Then if we want to check if last_pymnt_from_issue_cat could be a useful feature:
by.pymnt.gap <- with(loan, table(last_pymnt_from_issue_cat, loan_status))
by.pymnt.gap <- by.pymnt.gap[c("(-1,0]", "(0,92]", "(92,184]", "(184,275]", "(275,366]",
                               "(366,457]", "(457,549]", "(549,639]", "(639,730]", "no pymnt"),
                             c("Charged Off", "Default", "Late (31-120 days)",
                               "Late (16-30 days)", "In Grace Period", "Current",
                               "Fully Paid")]

In [47]:
# As seen below, 
by.pymnt.gap

                         loan_status
last_pymnt_from_issue_cat Charged Off Default Late (31-120 days)
                (-1,0]              0       0                  0
                (0,92]            939       0                  0
                (92,184]         2332       0                  0
                (184,275]        2744       0                  0
                (275,366]        2789       0                  0
                (366,457]        2772       1                  0
                (457,549]        1195       9               1625
                (549,639]         116       1               1409
                (639,730]           8       0                123
                no pymnt          148       0                  0
                         loan_status
last_pymnt_from_issue_cat Late (16-30 days) In Grace Period Current Fully Paid
                (-1,0]                    0               0       0        492
                (0,92]                    0          

In [48]:
# (-1, 0] cat means that the last payment date is issue date. And As you can see that 100% of this cat are fully paid.
# Within the first year, the loan is either fully paid or charged off.
# majority of the loan is in current category which is normal. 

round(100 * by.pymnt.gap / apply(by.pymnt.gap, 1, sum), 3)

                         loan_status
last_pymnt_from_issue_cat Charged Off Default Late (31-120 days)
                (-1,0]          0.000   0.000              0.000
                (0,92]         21.268   0.000              0.000
                (92,184]       36.506   0.000              0.000
                (184,275]      36.872   0.000              0.000
                (275,366]      33.250   0.000              0.000
                (366,457]      31.372   0.011              0.000
                (457,549]      13.171   0.099             17.910
                (549,639]       0.167   0.001              2.024
                (639,730]       0.042   0.000              0.644
                no pymnt      100.000   0.000              0.000
                         loan_status
last_pymnt_from_issue_cat Late (16-30 days) In Grace Period Current Fully Paid
                (-1,0]                0.000           0.000   0.000    100.000
                (0,92]                0.000          

In [49]:
# see if this feature has predictive power
loan$loan_status_binary <- as.factor(ifelse(loan$loan_status %in% c('Fully Paid', 'Current'), 'okay', 'past_due'))
mod2 <- glm(loan_status_binary ~ last_pymnt_from_issue, loan, family = 'binomial')
# consider use piecewise linear regression
summary(mod2)


Call:
glm(formula = loan_status_binary ~ last_pymnt_from_issue, family = "binomial", 
    data = loan)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2127  -0.4252  -0.4236  -0.3773   3.9866  

Coefficients:
                        Estimate Std. Error  z value Pr(>|z|)    
(Intercept)            8.259e-02  1.933e-02    4.274 1.92e-05 ***
last_pymnt_from_issue -4.014e-03  4.006e-05 -100.207  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 106836  on 133886  degrees of freedom
Residual deviance:  96924  on 133885  degrees of freedom
AIC: 96928

Number of Fisher Scoring iterations: 5


In [50]:
# the results look weird
# this is complete or more accurately quasi complete separatable data (no pymnt -> past_due, (-1, 0) -> okay).
# MLE for logistic regression doesn't exist in the case of complete or more accurately quasi.

loan$last_pymnt_from_issue_cat <- relevel(as.factor(loan$last_pymnt_from_issue_cat), ref = 'no pymnt')
loan$loan_status_binary <- as.factor(ifelse(loan$loan_status %in% c('Fully Paid', 'Current'), 'okay', 'past_due'))
mod1 <- glm(loan_status_binary ~ last_pymnt_from_issue_cat, loan, family = 'binomial')
summary(mod1)


Call:
glm(formula = loan_status_binary ~ last_pymnt_from_issue_cat, 
    family = "binomial", data = loan)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.9592  -0.3181  -0.3181  -0.1832   2.8619  

Coefficients:
                                   Estimate Std. Error z value Pr(>|z|)
(Intercept)                           15.57     119.63   0.130    0.896
last_pymnt_from_issue_cat(-1,0]      -31.13     136.45  -0.228    0.820
last_pymnt_from_issue_cat(0,92]      -16.87     119.63  -0.141    0.888
last_pymnt_from_issue_cat(184,275]   -16.10     119.63  -0.135    0.893
last_pymnt_from_issue_cat(275,366]   -16.26     119.63  -0.136    0.892
last_pymnt_from_issue_cat(366,457]   -16.35     119.63  -0.137    0.891
last_pymnt_from_issue_cat(457,549]   -16.36     119.63  -0.137    0.891
last_pymnt_from_issue_cat(549,639]   -18.52     119.63  -0.155    0.877
last_pymnt_from_issue_cat(639,730]   -19.64     119.63  -0.164    0.870
last_pymnt_from_issue_cat(92,184]    -16.12

In [51]:
with(loan, table(loan_status_binary, last_pymnt_from_issue_cat))

                  last_pymnt_from_issue_cat
loan_status_binary no pymnt (-1,0] (0,92] (184,275] (275,366] (366,457]
          okay            0    492   3476      4698      5599      6063
          past_due      148      0    939      2744      2789      2773
                  last_pymnt_from_issue_cat
loan_status_binary (457,549] (549,639] (639,730] (92,184]
          okay          6244     66172     18781     4056
          past_due      2829      3434       318     2332

In [52]:
# update mod1 so that we don't have no pymnt and (-1,0] categories.

mod1 <- glm(loan_status_binary ~ last_pymnt_from_issue_cat,
            subset(loan, !last_pymnt_from_issue_cat %in% c('no pymnt', '(-1,0]')), family = 'binomial')
summary(mod1)
# The results look okay now. 


Call:
glm(formula = loan_status_binary ~ last_pymnt_from_issue_cat, 
    family = "binomial", data = subset(loan, !last_pymnt_from_issue_cat %in% 
        c("no pymnt", "(-1,0]")))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.9592  -0.3181  -0.3181  -0.1832   2.8619  

Coefficients:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                        -1.30882    0.03678  -35.59   <2e-16 ***
last_pymnt_from_issue_cat(184,275]  0.77110    0.04393   17.55   <2e-16 ***
last_pymnt_from_issue_cat(275,366]  0.61192    0.04347   14.08   <2e-16 ***
last_pymnt_from_issue_cat(366,457]  0.52655    0.04334   12.15   <2e-16 ***
last_pymnt_from_issue_cat(457,549]  0.51712    0.04320   11.97   <2e-16 ***
last_pymnt_from_issue_cat(549,639] -1.64971    0.04073  -40.50   <2e-16 ***
last_pymnt_from_issue_cat(639,730] -2.76973    0.06745  -41.06   <2e-16 ***
last_pymnt_from_issue_cat(92,184]   0.75535    0.04503   16.77   <2e-16 ***
---
S

In [53]:
# Next，we can treat all date features in the same way as above. 

# Find date related features.
date.cols <- colnames(loan)[c(which(grepl('_d$', colnames(loan))),
                              which(grepl('_date$', colnames(loan))))]
# "issue_d"                   "last_pymnt_d"              "next_pymnt_d"             
# "last_credit_pull_d"        "hardship_start_date"       "hardship_end_date"        
# "payment_plan_start_date"   "debt_settlement_flag_date" "settlement_date"  

# update these features to R supported date format
for (col_i in date.cols) {
  loan[, col_i] <-  as.Date(as.yearmon(loan[, col_i], "%b-%Y"))
}

In [54]:
# dates are not helpful when predicting, we will need to transform them into length
loan$mths_since_issue <- as.integer((as.Date('2017-11-01') - loan$issue_d) /30)
# See if recently there is a credit pull which implies a big event like new loan -> may affect current payment
loan$mths_since_last_credit_pull <- as.integer((as.Date('2017-11-01') - loan$last_credit_pull_d) /30)

In [55]:
# Define a function to turn current dates column into a new category column (since issue date). 
# This uses the same methodology as last_pymnt_since_issue.

TransformToLengthFromIssueDate <- function(loan, col.name, new.col.name, other.level) {
  # get difference in months.
  loan[, new.col.name] <-
    ifelse(is.na(loan[, col.name]), other.level,
           as.character(cut(as.integer((loan[, col.name] - loan$issue_d) /30), 
                            c(min(as.integer((loan[, col.name] - loan$issue_d) /30), na.rm = T) - 1,
                              quantile(as.integer((loan[, col.name] - loan$issue_d) /30), c(0.1, 0.9), na.rm = T),
                              max(as.integer((loan[, col.name] - loan$issue_d) /30), na.rm = T)))))
  return(loan)
}

In [56]:
# use the function to transform date columns
loan <- TransformToLengthFromIssueDate(loan, 'hardship_start_date' ,'hardship_since_issue', 'no_hs')
loan <- TransformToLengthFromIssueDate(loan, 'settlement_date' ,'settlement_since_issue', 'no_settle')
# Drop original dates columns
loan <- loan[, -which(colnames(loan) %in% date.cols)]

In [57]:
# see how many unique values
num.value <- sapply(loan, function(x){return(length(unique(x)))})
# find out column is string, and unique values > 50
colnames(loan)[intersect(which(sapply(loan, function(x){return(is.character(x))})), 
                         which(num.value >= 50))]

[1] "id"               "int_rate"         "emp_title"        "url"             
[5] "zip_code"         "addr_state"       "earliest_cr_line" "revol_util"

In [58]:
# features such as ID has no predictive power, 
# features like int_rate, revol_util are string due to % sign. change these features to numetical.
head(loan$int_rate)
which(sapply(loan[1, ], function(x){return(grepl('%', x))}))
loan$int_rate <- as.numeric(sapply(strsplit(loan$int_rate, '%'), '[', 1))
loan$revol_util <- as.numeric(sapply(strsplit(loan$revol_util, '%'), '[', 1))

[1] "  5.32%" "  9.75%" " 15.31%" " 15.31%" " 12.99%" " 11.99%"

int_rate revol_util 
         7         35

In [59]:
head(loan$int_rate)

[1]  5.32  9.75 15.31 15.31 12.99 11.99

In [60]:
# earliest_cr_line is date feature, transfer it to mths_since_crline
loan$earliest_cr_line <-  as.Date(as.yearmon(loan$earliest_cr_line, "%b-%Y"))
loan$mths_since_crline <- as.integer((as.Date('2017-11-01') - loan$earliest_cr_line) /30)

In [61]:
# For categorical features with too many levels, could collapse levels as we did before.
feat.w.many.levels <- colnames(loan)[intersect(which(sapply(loan, function(x) {
  return(is.character(x))})),
  which(num.value >= 50))]

In [62]:
# drop these features with too many levels.
feat.w.many.levels
loan <- loan[, -which(colnames(loan) %in% c(names(which(num.value == 1)), feat.w.many.levels))]

[1] "id"         "emp_title"  "url"        "zip_code"   "addr_state"

In [ ]:
# save.image()

In [4]:
# Update features to reflect loan is jointly applied

colnames(loan)[which(grepl('joint', colnames(loan)))]
loan$dti <- ifelse(!is.na(loan$dti_joint), loan$dti_joint, loan$dti)
loan$annual_inc <- ifelse(!is.na(loan$annual_inc_joint), loan$annual_inc_joint, loan$annual_inc)
loan$verification_status <- ifelse(!is.na(loan$verification_status_joint), loan$verification_status_joint, loan$verification_status)

loan <- loan[, -which(grepl('joint', colnames(loan)))]

[1] "annual_inc_joint"          "dti_joint"                
[3] "verification_status_joint"

In [5]:
# dealing with missing value

# top 10 NA columns 
num.NA <- sort(sapply(loan, function(x) { sum(is.na(x))} ), decreasing = TRUE)
sort(num.NA, decreasing = TRUE)[1:10]

orig_projected_additional_accrued_interest 
                                    133252 
                             deferral_term 
                                    133113 
                           hardship_amount 
                                    133113 
                           hardship_length 
                                    133113 
                              hardship_dpd 
                                    133113 
            hardship_payoff_balance_amount 
                                    133113 
              hardship_last_payment_amount 
                                    133113 
                         settlement_amount 
                                    132800 
                     settlement_percentage 
                                    132800 
                           settlement_term 
                                    132800

### From above exploration, we can find that most of the NAs come from hardship and settlement related features.
Take a look at the feature dictionary, we found that deferrarl_term is definded as "Amount of months that the borrower is expected to pay less than the contractual monthly payment amount due to a hardship plan!" which is inside hardship feature.

From lending club website:  
Hardship plans allow borrowers to temporarily make interest-only payments to accommodate an unexpected life event. 

Debt_Settlement_Flag:  
whether or not the borrower, who has charged-off, is working with a debt-settlement company.

Thus, not all users need these two plans, but these two plans indeed give us more information about the condition of a loan. So even with lots of missing data, I want to keep as much information as possible.

In [6]:
# check columns with a lot of NA, for example, hardship related features

colnames(loan)[which(grepl('hardship', colnames(loan)))]
summary(loan$orig_projected_additional_accrued_interest)
loan$orig_projected_additional_accrued_interest[which(is.na(loan$orig_projected_additional_accrued_interest))] <- 0

[1] "hardship_flag"                  "hardship_type"                 
 [3] "hardship_reason"                "hardship_status"               
 [5] "hardship_amount"                "hardship_length"               
 [7] "hardship_dpd"                   "hardship_loan_status"          
 [9] "hardship_payoff_balance_amount" "hardship_last_payment_amount"  
[11] "hardship_since_issue"

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  22.92  187.94  357.51  451.39  606.12 1889.97  133252 

In [7]:
# check some columns and find out not only NA but also empty value.
# not all users will use hardship program provided by lending club

loan$hardship_reason <- ifelse(loan$hardship_reason == '', 'no_hs', loan$hardship_reason)
loan$hardship_status <- ifelse(loan$hardship_status == '', 'no_hs', loan$hardship_status)
loan$hardship_loan_status <- ifelse(loan$hardship_loan_status == '', 'no_hs', loan$hardship_loan_status)
loan$hardship_amount[which(is.na(loan$hardship_amount))] <- 0
loan$hardship_dpd[which(is.na(loan$hardship_dpd))] <- 0
loan$hardship_payoff_balance_amount[which(is.na(loan$hardship_payoff_balance_amount))] <- 0
loan$hardship_last_payment_amount[which(is.na(loan$hardship_last_payment_amount))] <- 0

loan <- loan[, -which(colnames(loan) %in% c('deferral_term',
                                            'hardship_length', 'hardship_type'))]

num.empty <- sapply(loan[, colnames(loan)[which(sapply(loan, function(x){return(is.character(x))}))]],
                    function(x){return(length(which(x == "")))})
num.empty[which(num.empty > 0)]
loan <- loan[, -which(colnames(loan) %in% c('verification_status', 'desc', 'title'))] # loan title is not important, will deal with settlement later

verification_status                desc               title   settlement_status 
             131226              133877               10621              132800

In [8]:
colnames(loan)[which(grepl('settlement', colnames(loan)))]

[1] "debt_settlement_flag"   "settlement_status"      "settlement_amount"     
[4] "settlement_percentage"  "settlement_term"        "settlement_since_issue"

In [9]:
# Similarly for settlement

loan$settlement_amount[which(is.na(loan$settlement_amount))] <- 0
loan$settlement_percentage[which(is.na(loan$settlement_percentage))] <- 0
loan$settlement_term[which(is.na(loan$settlement_term))] <- 0
loan$settlement_status <- ifelse(is.na(loan$settlement_status), 'no_settlement', loan$settlement_status)

num.NA <- sort(sapply(loan, function(x) { sum(is.na(x))} ), decreasing = TRUE)
sort(num.NA, decreasing = TRUE)[1:10]

mths_since_last_record       mths_since_recent_bc_dlq 
                        110216                         100131 
   mths_since_last_major_derog mths_since_recent_revol_delinq 
                         95813                          86104 
        mths_since_last_delinq                        il_util 
                         64565                          17879 
         mths_since_recent_inq               num_tl_120dpd_2m 
                         13798                           7621 
            mths_since_rcnt_il             mo_sin_old_il_acct 
                          3701                           3646

In [10]:
# deal with mths_since features with NAs.

for(col_i in setdiff(names(num.NA)[which(grepl('mths_since', names(num.NA))& num.NA > 0)],
                     c('mths_since_issue', 'mths_since_crline', 'mths_since_last_credit_pull'))) {
  breaks <- quantile(loan[, col_i], c(0.1, 0.5, 0.9), na.rm = T)
  breaks <- c(min(loan[, col_i], na.rm = T) - 1, breaks, max(loan[, col_i], na.rm = T))
  loan[, col_i] <- ifelse(is.na(loan[, col_i]),
                          'not_avail', as.character(cut(loan[, col_i], breaks = breaks)))
}

In [11]:
# check NA again

num.NA <- sort(sapply(loan, function(x) { sum(is.na(x))} ), decreasing = TRUE)
sort(num.NA, decreasing = TRUE)[which(num.NA > 0)]

il_util            num_tl_120dpd_2m 
                      17879                        7621 
         mo_sin_old_il_acct                     bc_util 
                       3646                        1175 
           percent_bc_gt_75              bc_open_to_buy 
                       1129                        1111 
             last_pymnt_d_1             last_pymnt_year 
                        148                         148 
             last_pymnt_mon                  revol_util 
                        148                          71 
                   all_util                 open_acc_6m 
                         67                          62 
                total_cu_tl                inq_last_12m 
                         62                          62 
                open_act_il                 open_il_12m 
                         61                          61 
                open_il_24m                total_bal_il 
                         61                          61 
                open_rv_12m                 open_rv_24m 
                         61                          61 
                 max_bal_bc                      inq_fi 
                         61                          61 
mths_since_last_credit_pull              inq_last_6mths 
                          2                           1

In [12]:
# Deal with il_util feature. From the dictionary, 
# il_util defined as : Ratio of total current balance to high credit/credit limit on all install acct
# Thus il_util = total_bal_il / total_il_high_credit_limit

# il_util is na may due to no open account (no total_balance_il). 
summary(subset(loan, is.na(il_util))$open_act_il)
summary(subset(loan, is.na(il_util))$total_bal_il)
head(loan[which(is.na(loan$open_act_il)),
          c('il_util', 'total_bal_il', 'total_il_high_credit_limit')])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
 0.0000  0.0000  0.0000  0.2802  0.0000  8.0000      61 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
      0       0       0   12246       0  573170      61 

,il_util,total_bal_il,total_il_high_credit_limit
133685,NA,NA,0
133744,NA,NA,107007
133757,NA,NA,20681
133780,NA,NA,57524
133788,NA,NA,0
133825,NA,NA,15075


In [13]:
# Simiarly for the total_il_high_credit_limit is NA. 
with(subset(loan, is.na(il_util)), summary(total_il_high_credit_limit))

# Lastly, il_util is NA, but balance & credit_limit are not NA.
head(loan[which(is.na(loan$il_util) & loan$total_il_high_credit_limit != 0),
          c('il_util', 'total_bal_il', 'total_il_high_credit_limit')])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      0       0       0   15053       0  808529 

,il_util,total_bal_il,total_il_high_credit_limit
32,NA,18037,38597
49,NA,29852,35854
75,NA,65278,75300
80,NA,30356,36715
91,NA,238537,243588
150,NA,41240,42525


In [14]:
# update il_util

loan$il_util <- ifelse(is.na(loan$il_util) & loan$total_il_high_credit_limit != 0, 
                       loan$total_bal_il/ loan$total_il_high_credit_limit, loan$il_util)
summary(subset(loan, is.na(il_util) & total_il_high_credit_limit == 0)$open_act_il)
loan$il_util <-  ifelse(is.na(loan$il_util), 'no_il',
                        as.character(cut(loan$il_util, 
                                         c(min(loan$il_util, na.rm = T) - 0.01,
                                           quantile(loan$il_util, na.rm = T, c(0.1, 0.9)),
                                           max(loan$il_util, na.rm = T)))))
table(loan$il_util)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
      0       0       0       0       0       0      11 


(-0.01,32]    (32,96]   (96,558]      no_il 
     12318      97285      10636      13648 

In [15]:
# Similarly to mo_sin_old_il_acct

loan$mo_sin_old_il_acct <-  ifelse(is.na(loan$mo_sin_old_il_acct), 'no_il',
                                   as.character(cut(loan$mo_sin_old_il_acct, 
                                                    c(min(loan$mo_sin_old_il_acct, na.rm = T) - 0.01,
                                                      quantile(loan$mo_sin_old_il_acct, na.rm = T, c(0.1, 0.9)),
                                                      max(loan$mo_sin_old_il_acct, na.rm = T)))))

In [16]:
# Now num_tl_120dpd_2m feature. 
# According to the dictionary, num_tl_120dpd_2m: Number of accounts currently 120 days past due (updated in past 2 months)

summary(subset(loan, is.na(num_tl_120dpd_2m))$open_acc) # just to make sure that the account is not NA.
with(subset(loan, is.na(num_tl_120dpd_2m)), summary(num_tl_30dpd))
loan$num_tl_120dpd_2m <- ifelse(is.na(loan$num_tl_120dpd_2m), 0, loan$num_tl_120dpd_2m)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    1.0     9.0    12.0    13.3    17.0    51.0 

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.000000 0.000000 0.000000 0.005773 0.000000 2.000000 

In [17]:
# the rest ones, NA is not significant. Use median to fill NA.

num.NA <- sort(sapply(loan, function(x) { sum(is.na(x))} ), decreasing = TRUE)
for(col_i in names(num.NA)[num.NA > 0]) {
  loan[, col_i] <- ifelse(is.na(loan[, col_i]), median(loan[, col_i], na.rm = T), loan[, col_i])
}

In [18]:
dim(loan)

[1] 133887    126

In [19]:
# There are still a lot of features. Use t-test and p-value to select numeric features.

# add and drop some features. 
loan <- loan[, -which(colnames(loan) %in% c('grade', 'int_rate', 'sub_grade', 'policy_code'))]
# fully paid and current -> Okay, otherwise -> past_due
loan$loan_status_binary <- as.factor(ifelse(loan$loan_status %in% c('Fully Paid', 'Current'), 'okay', 'past_due'))
loan$loan_status_binary <- relevel(loan$loan_status_binary, ref = "okay")

numeric.feats <- colnames(loan)[which(sapply(loan, function(x){return(is.numeric(x))}))]
for(col_i in numeric.feats) {
  formula = paste(col_i, " ~ loan_status_binary")
  p.val <- t.test(as.formula(formula), data = loan)$p.value
  if(p.val >= 0.05) { # t-test 95% confidence internal not significant
    loan[, col_i] <- NULL
  }
}

In [23]:
dim(loan)

[1] 133887    115

In [24]:
# Similarly, use chi-squre to select category features

cat.feats <- colnames(loan)[which(sapply(loan, function(x){return(is.character(x))}))]
cat.feats <- setdiff(cat.feats, 'loan_status_binary')
for(col_i in cat.feats) {
  p.val <- chisq.test(x = loan[, col_i], y = loan$loan_status_binary)$p.value
  if(p.val >= 0.05) {
    loan[, col_i] <- NULL
  }
}

Warning message in chisq.test(x = loan[, col_i], y = loan$loan_status_binary):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(x = loan[, col_i], y = loan$loan_status_binary):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(x = loan[, col_i], y = loan$loan_status_binary):
“Chi-squared approximation may be incorrect”

In [25]:
dim(loan)

[1] 133887    113

In [28]:
# deal with fico features, no need to have too many features.

loan$fico_range_high <- NULL
loan$fico_range_low <- NULL
loan$last_fico <- with(loan, (last_fico_range_high + last_fico_range_low)/2)
loan$last_fico_range_high <- loan$last_fico_range_low <- NULL
loan$emp_length <- ifelse(loan$emp_length == 'n/a', loan$emp_length,
                          ifelse(loan$emp_length %in% c('< 1 year', '1 year', '2 years', '3 years'),
                                 '< 3 years', ifelse(loan$emp_length %in% c('4 years', '5 years', '6 years', '7 years'), 
                                                     '4-7 years', '> 8 years')))

In [29]:
# save data to local csv file for backup.
# write.csv(loan ,'loan_feature_fin.csv')

### Build classification models using the data above.  
1. Random forest trees  
2. SVM  
3. Logistic regression using lib glmnet.


### Split data into training and testing sets. And normalize all training data.

In [ ]:
set.seed(1)
train.ind <- sample(1:dim(loan)[1], 0.7* dim(loan)[1])
train <- loan[train.ind, ]
test <- loan[-train.ind, ]

numeric.feats <- colnames(loan)[which(sapply(loan, function(x){return(is.numeric(x))}))]
train.sub <- train
train.sub.scale <- train.sub
# normalize input numerical features
for (col_i in numeric.feats){
  train.sub.scale[, c(col_i)] <- scale(train.sub.scale[, c(col_i)])
}
# normalize testing data
test.sub <- test
test.sub.scale <- test.sub
for (col_i in numeric.feats){
  test.sub.scale[, c(col_i)] <- scale(test.sub.scale[, c(col_i)])
}

### Random Forest Trees

In [ ]:
require(randomForest)
library(pROC)
ind.rf = train.sub.scale[, -which(colnames(train) %in% c('loan_status_binary', 'loan_status'))]
colnames(ind.rf)
dep <- train$loan_status_binary
dep.test <- test.sub.scale$loan_status_binary

ind.rf2 <- ind.rf
ind.rf2 <- as.data.frame(unclass(ind.rf2))

rf.mod = randomForest(dep[1:10000] ~ ., data=ind.rf2[1:10000, ], ntree = 200, importance=TRUE)

ind.test.rf = test.sub.scale[, -which(colnames(train) %in% c('loan_status_binary', 'loan_status'))]
ind.test.rf <- as.data.frame(unclass(ind.test.rf))
pred.rf <- predict(rf.mod, ind.test.rf[1:10000,], type="prob")
pred.rf[,1][1:10]
res.rf <- roc(dep.test[1:10000], pred.rf[,1])
dep.test[1:10]
plot(res.rf)
auc(res.rf)

The ROC curve looks like below with area under curve (AUC) = 0.9822
![RF ROC](./RF_confusion_matrix.png)

### SVM

In [ ]:
# SVM
library("e1071")
svm.model <- svm(dep[1:10000] ~ ., data = ind.rf2[1:10000, ], 
                 method="C-classification", kernel="radial", probability=TRUE)
svm.model
pred.svm1 <- predict(svm.model, ind.test.rf[1:10000,], probability=TRUE)
head(pred.svm1)
table(pred.svm1, dep.test[1:10000])
# res.svm1 <- roc(dep.test[1:10000], pred.svm1)

Call:
svm(formula = dep[1:10000] ~ ., data = ind.rf2[1:10000, 
    ], probability = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 
      gamma:  0.005347594 

Number of Support Vectors:  1386


pred.svm1  okay past_due
  okay     8562      104
  past_due   50     1284

### Logistic regression using Glmnet

In [ ]:
library(glmnet)
# binary model
ind <- sparse.model.matrix( ~. , train.sub.scale[, -which(colnames(train) %in% c('loan_status_binary', 'loan_status'))])
dep <- train$loan_status_binary
cv.mod <- cv.glmnet(ind[1:10000, ], dep[1:10000], family = 'binomial')
plot(cv.mod)


![glmnet lambda](./glmnet_lambda1.png)

In [ ]:
ind.test <- sparse.model.matrix( ~. , test.sub.scale[, -which(colnames(train) %in% c('loan_status_binary', 'loan_status'))])
dep.test <- test.sub.scale$loan_status_binary
pred <- predict(cv.mod, ind.test[1:10000,])
res <- roc(dep.test[1:10000], pred)
auc(res)
plot(res)

AUC = 0.9919
![ROC_glmnet](./ROC_glmnet.png)

#### Training with all data using glmnet. 

In [ ]:
# tain on all data
cv2.mod <- cv.glmnet(ind, dep, family = 'binomial')
plot(cv2.mod)

pred2 <- predict(cv2.mod, ind.test)
res2 <- roc(dep.test, pred2)
auc(res2)
plot(res2)

Lambda
![lambda](./glmnet_lambda2.png)
AUC = 0.9931
![ROC glmnet2](./ROC_full_data.png)